In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

In [2]:
train_dir = r'F:\programming part\deep_learning\datasets\face_mask_detection\train'
test_dir = r'F:\programming part\deep_learning\datasets\face_mask_detection\test'
validation_dir = r'F:\programming part\deep_learning\datasets\face_mask_detection\validation'

In [3]:
from tensorflow.keras.applications import VGG19

In [4]:
conv_base = VGG19(weights = 'imagenet',include_top = False,input_shape = (128,128,3))

80142336/80134624 [==============================] - 25s 0us/step


In [5]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [6]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256,activation = 'relu'))
model.add(layers.Dense(1,activation = 'sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 22,122,049
Trainable params: 22,122,049
Non-trainable params: 0
_________________________________________________________________


In [4]:
model = models.load_model("Mask_model_2-01.h5")

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')


Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


path, dirs, files = next(os.walk(train_dir))
train_count = len(files)

path, dirs, files = next(os.walk(validation_dir))
validation_count = len(files)

In [6]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("Mask_model_2-{epoch:02d}.h5")

In [7]:
model.compile(loss = 'binary_crossentropy',optimizer = optimizers.RMSprop(learning_rate = 2e-5),metrics = ['acc'])

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 312,
    epochs = 5,
    validation_data = validation_generator,
    validation_steps = 25,
    callbacks = [checkpoint_cb]
)

C:\Users\amit\.conda\envs\myenv\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
312/312 [==============================] - 2700s 9s/step - loss: 0.0201 - acc: 0.9964 - val_loss: 0.0031 - val_acc: 1.0000
Epoch 2/5
 38/312 [==>...........................] - ETA: 36:02 - loss: 0.0069 - acc: 0.9975

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca()
plt.show()

In [ ]:
model.save("final_face_detection_model.h5")

In [ ]:
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [6]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=10,
        class_mode='binary')

Found 992 images belonging to 2 classes.


In [7]:
model.evaluate_generator(test_generator, steps = 20)

C:\Users\amit\.conda\envs\myenv\lib\site-packages\tensorflow\python\keras\engine\training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.0008292501443065703, 1.0]

Thank you!!!